输入输出通常可以划分为几个大类：读取文本文件和其他更高效的磁盘存储格式，加载数据库中的数据，利用Web API操作网络资源。

# 6.1 读写文本格式的数据-----------------------------------------------------------------------------

表6-1对它们进行了总结，其中read_csv和read_table可能会是你今后用得最多的。

我将大致介绍一下这些函数在将文本数据转换为DataFrame时所用到的一些技术。这些函数的选项可以划分为以下几个大类：

> * 索引：将一个或多个列当做返回的DataFrame处理，以及是否从文件、用户获取列名。
> * 类型推断和数据转换：包括用户定义值的转换、和自定义的缺失值标记列表等。
> * 日期解析：包括组合功能，比如将分散在多个列中的日期时间信息组合成结果中的单个列。
> * 迭代：支持对大文件进行逐块迭代。
> * 不规整数据问题：跳过一些行、页脚、注释或其他一些不重要的东西（比如由成千上万个逗号隔开的数值数据）。


首先我们来看一个以逗号分隔的（CSV）文本文件：examples/ex1.csv

由于该文件以逗号分隔，所以我们可以使用read_csv将其读入一个DataFrame：

我们还可以使用read_table，**sep参数**指定分隔符：


【pd.readd_csv默认把第一行当作标题行】并不是所有文件都有标题行。看看下面这个文件：examples/ex2.csv

读入该文件的办法有两个。你可以让pandas为其分配默认的列名，也可以自己定义列名。使用**header参数**或者**names参数**：

In [1]:
import pandas as pd
pd.read_csv(r'examples\ex2.csv')

,1,2,3,4,hello
0,5,6,7,8,world
1,9,10,11,12,foo


In [2]:
pd.read_csv(r'examples\ex2.csv', header=None)

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [3]:
pd.read_csv(r'examples\ex2.csv', names = ['a', 'b', 'c', 'd', 'message'])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


假设你希望将message列做成DataFrame的索引。你可以明确表示要将该列放到索引的位置上，也可以通过**index_col参数**指定"message"：

In [4]:
names = ['a', 'b', 'c', 'd', 'message']
pd.read_csv(r'examples\ex2.csv', names=names, index_col='message')

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


如果希望将多个列做成一个**层次化索引**，只需传入由列编号或列名组成的列表即可：index_col=[...]

In [5]:
parsed = pd.read_csv('examples/csv_mindex.csv',
   ....:                      index_col=['key1', 'key2'])
parsed

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

有些情况下，有些表格可能不是用固定的分隔符去分隔字段的（比如空白符或其它模式）。看看下面这个文本文件：examples/ex3.txt

虽然可以手动对数据进行规整，这里的字段是被数量不同的空白字符间隔开的。

这种情况下，你可以传递一个正则表达式作为read_table的分隔符。**sep=re表达式**

这里，由于列名比数据行的数量少，所以read_table推断第一列应该是DataFrame的索引。

In [6]:
result = pd.read_table('examples/ex3.txt', sep='\s+')
result

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


这些解析器函数还有许多参数可以帮助你处理各种各样的异形文件格式（表6-2列出了一些）。比如说，你可以用**skiprows参数**跳过文件的第一行、第三行和第四行：

缺失值处理是文件解析任务中的一个重要组成部分。缺失数据经常是要么没有（空字符串），要么用某个标记值表示。默认情况下，pandas会用一组经常出现的标记值进行识别，比如NA及NULL：

**na_values参数**可以用一个列表或集合的字符串表示缺失值：

使用各列对应的缺失值，**na_values=字典**，可以实现不同的NA标记值。

In [7]:
result = pd.read_csv('examples/ex5.csv')
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [8]:
result = pd.read_csv('examples/ex5.csv', na_values=['NULL'])
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [9]:
sentinels = {'message': ['foo', 'NA'], 'something': ['two']}
pd.read_csv('examples/ex5.csv', na_values=sentinels)

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,NaN,5,6,NaN,8,world
2,three,9,10,11.0,12,NaN


> 表6-2列出了pandas.read_csv和pandas.read_table常用的选项。

# 逐块读取文本文件

在处理很大的文件时，或找出大文件中的参数集以便于后续处理时，你可能只想读取文件的一小部分或逐块对文件进行迭代。

在看大文件之前，我们先设置pandas显示地更紧些：**pd.options.display.max_rows = 10**

In [10]:
pd.options.display.max_rows = 10

result = pd.read_csv('examples/ex6.csv')
result

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q
...,...,...,...,...,...
9995,2.311896,-0.417070,-1.409599,-0.515821,L
9996,-0.479893,-0.650419,0.745152,-0.646038,E
9997,0.523331,0.787112,0.486066,1.093156,K
9998,-0.362559,0.598894,-1.843201,0.887292,G


如果只想读取几行（避免读取整个文件），通过**nrows**进行指定即可：

要逐块读取文件，可以指定**chunksize（行数）**：read_csv所返回的这个TextParser对象使你可以根据chunksize对文件进行逐块迭代。比如说，我们可以迭代处理ex6.csv，将值计数聚合到"key"列中，如下所示：

In [11]:
chunker = pd.read_csv('examples/ex6.csv', chunksize=1000)
chunker

In [12]:
chunker = pd.read_csv('examples/ex6.csv', chunksize=1000)
tot = pd.Series([])
for piece in chunker:
    tot  = tot.add(piece['key'].value_counts(), fill_value=0)
tot = tot.sort_values(ascending=False)
tot

E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
     ...  
5    157.0
2    152.0
0    151.0
9    150.0
1    146.0
Length: 36, dtype: float64

# 将数据写出到文本格式

数据也可以被输出为分隔符格式的文本。我们再来看看之前读过的一个CSV文件：examples/ex5.csv

利用DataFrame的to_csv方法，我们可以将数据写到一个以逗号分隔的文件中：

当然，还可以使用其他分隔符（由于这里直接写出到sys.stdout，所以仅仅是打印出文本结果而已）**sep='|'**

缺失值在输出结果中会被表示为空字符串。你可能希望将其表示为别的标记值：**na_rep='NULL'**

如果没有设置其他选项，则会写出行和列的标签。当然，它们也都可以被禁用：**index=False, header=False**

此外，你还可以只写出一部分的列，并以你指定的顺序排列：**columns=[...]**

Series也有一个to_csv方法：

In [13]:
data = pd.read_csv('examples/ex5.csv')
data

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [14]:
data.to_csv('examples/out.csv')

In [15]:
data.to_csv('examples/out.csv', index=False, header=False)

In [16]:
import sys
data.to_csv(sys.stdout, index=False, columns=['a', 'b', 'c'])

a,b,c
1,2,3.0
5,6,
9,10,11.0


# 处理分隔符格式

大部分存储在磁盘上的表格型数据都能用pandas.read_table进行加载。然而，有时还是需要做一些手工处理。由于接收到含有畸形行的文件而使read_table出毛病的情况并不少见。
为了说明这些基本工具，看看下面这个简单的CSV文件：examples/ex7.csv

对于任何单字符分隔符文件，可以直接使用Python内置的csv模块。将任意已打开的文件或文件型的对象传给csv.reader：

对这个reader进行迭代将会为每行产生一个元组（并移除了所有的引号）：对这个reader进行迭代将会为每行产生一个元组（并移除了所有的引号）：

现在，为了使数据格式合乎要求，你需要对其做一些整理工作。我们一步一步来做。首先，读取文件到一个多行的列表中：

然后，我们将这些行分为标题行和数据行：

然后，我们可以用**字典构造式和zip(\*values)**，后者将行转置为列，创建数据列的字典：

In [17]:
import csv
with open('examples/ex7.csv') as f:
    lines = list(csv.reader(f))
lines

[['a', 'b', 'c'], ['1', '2', '3'], ['1', '2', '3']]

In [18]:
header,values = lines[0], lines[1:]
data_dict = {h:v for h,v in zip(header, zip(*values))}
data_dict

{'a': ('1', '1'), 'b': ('2', '2'), 'c': ('3', '3')}

CSV文件的形式有很多。只需**定义csv.Dialect子类**即可定义出新格式（如专门的分隔符、字符串引用约定、行结束符等）：

各个CSV语支的参数也可以用关键字的形式提供给csv.reader，赋给**dialect参数**而无需定义子类：

In [19]:
class my_dialect(csv.Dialect):
    lineterminator = '\n'
    delimiter = ';'
    quotechar = '"'
    quoting = csv.QUOTE_MINIMAL

f = open('examples/ex7.csv')
reader = csv.reader(f, dialect=my_dialect)
for line in reader:
    print(line)

['a,"b","c"']
['1,"2","3"']
['1,"2","3"']


> 可用的选项（csv.Dialect的属性）及其功能如表6-3所示。

> delimiter分隔符  | lineterminator行结束符 | quotechar字段引用符 | quoting引用约定 | skiinitialspace | doublequote | escapechar

要手工输出分隔符文件，你可以使用csv.writer。它接受一个已打开且可写的文件对象以及跟csv.reader相同的那些语支和格式化选项：

# JSON数据

它是一种比表格型文本格式（如CSV）灵活得多的数据格式。

许多Python库都可以读写JSON数据。我将使用json，因为它是构建于Python标准库中的。通过json.loads即可将JSON字符串转换成Python形式：

In [20]:
obj = """
{"name": "Wes",
 "places_lived": ["United States", "Spain", "Germany"],
 "pet": null,
 "siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]},
              {"name": "Katie", "age": 38,
               "pets": ["Sixes", "Stache", "Cisco"]}]
}
"""

In [21]:
import json
result = json.loads(obj)
result

{'name': 'Wes',
 'pet': None,
 'places_lived': ['United States', 'Spain', 'Germany'],
 'siblings': [{'age': 30, 'name': 'Scott', 'pets': ['Zeus', 'Zuko']},
  {'age': 38, 'name': 'Katie', 'pets': ['Sixes', 'Stache', 'Cisco']}]}

json.dumps则将Python对象转换成JSON格式

In [22]:
asjson = json.dumps(result)
asjson

'{"name": "Wes", "places_lived": ["United States", "Spain", "Germany"], "pet": null, "siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]}, {"name": "Katie", "age": 38, "pets": ["Sixes", "Stache", "Cisco"]}]}'

如何将（一个或一组）JSON对象转换为DataFrame或其他便于分析的数据结构就由你决定了。

最简单方便的方式是：向DataFrame构造器传入一个字典的列表（就是原先的JSON对象），并选取数据字段的子集：

In [23]:
siblings = pd.DataFrame(result['siblings'], columns=['name','age'])
siblings

,name,age
0,Scott,30
1,Katie,38


pandas.read_json可以自动将**特别格式的JSON**数据集转换为Series或DataFrame。例如：examples/example.json

**pandas.read_json**的默认选项假设JSON数组中的每个对象是表格中的一行：

In [24]:
data = pd.read_json('examples/example.json')
data

,a,b,c
0,1,2,3
1,4,5,6
2,7,8,9


# XML和HTML：Web信息收集

Python有许多可以读写常见的HTML和XML格式数据的库，包括lxml、Beautiful Soup和html5lib。lxml的速度比较快，但其它的库处理有误的HTML或XML文件更好。

pandas有一个内置的功能**read_html**，它可以使用lxml和Beautiful Soup自动将HTML文件中的表格解析为DataFrame对象。

### 利用lxml.objectify解析XML

XML（Extensible Markup Language）是另一种常见的支持分层、嵌套数据以及元数据的结构化数据格式。本书所使用的这些文件实际上来自于一个很大的XML文档。

前面，我介绍了pandas.read_html函数，它可以使用lxml或Beautiful Soup从HTML解析数据。XML和HTML的结构很相似，但XML更为通用。这里，我会用一个例子演示如何利用lxml从XML格式解析数据。

# 6.2 二进制数据格式-----------------------------------------------------------------------------------

实现数据的高效二进制格式存储最简单的办法之一是使用Python内置的pickle序列化。

pandas对象都有一个用于将数据以pickle格式保存到磁盘上的**to_pickle方法**：

你可以通过pickle直接读取被pickle化的数据，或是使用更为方便的pandas.read_pickle：

In [25]:
frame = pd.read_csv('examples/ex1.csv')
frame.to_pickle('examples/frame_pickle')

pd.read_pickle('examples/frame_pickle')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


注意：pickle仅建议用于短期存储格式。其原因是很难保证该格式永远是稳定的；今天pickle的对象可能无法被后续版本的库unpickle出来。

pandas内置支持两个二进制数据格式：HDF5和MessagePack。下一节，我会给出几个HDF5的例子，但我建议你尝试下不同的文件格式，看看它们的速度以及是否适合你的分析工作。pandas或NumPy数据的其它存储格式有：

bcolz：一种可压缩的列存储二进制格式，基于Blosc压缩库。

Feather：我与R语言社区的Hadley Wickham设计的一种跨语言的列存储文件格式。Feather使用了Apache Arrow的列式内存格式。

# 使用HDF5格式

虽然可以用PyTables或h5py库直接访问HDF5文件，pandas提供了更为高级的接口，可以简化存储Series和DataFrame对象。

**HDFStore**类可以像字典一样，处理低级的细节：

HDF5文件中的对象可以通过与字典一样的API进行获取：

In [26]:
import numpy as np
frame = pd.DataFrame({'a': np.random.randn(100)})
store = pd.HDFStore('mydata.h5')

In [27]:
store['obj1'] = frame
store['obj1_col'] = frame['a']
store['obj1']

,a
0,0.550216
1,1.885966
2,-1.135988
3,2.351502
4,-0.748439
...,...
95,-0.057352
96,0.454569
97,1.610045
98,0.394329


HDFStore支持两种存储模式**fromat参数**，'fixed'和'table'。后者通常会更慢，但是支持使用特殊语法进行查询操作：

put方法是store['obj2'] = frame方法的显示版本，允许我们设置其它的选项，比如格式。

In [28]:
store.put('obj2', frame, format='table')
store

<class 'pandas.io.pytables.HDFStore'>
File path: mydata.h5
/obj1                frame        (shape->[100,1])                                       
/obj1_col            series       (shape->[100])                                         
/obj2                frame_table  (typ->appendable,nrows->100,ncols->1,indexers->[index])
/obj3                frame        (shape->[100,1])                                       

In [29]:
store.select('obj2', where=['index >= 10 and index <= 15'])

,a
10,-0.982180
11,0.436118
12,-0.087461
13,-0.108770
14,-0.577035
15,-1.017190


pandas.read_hdf函数可以快捷使用这些工具：

In [30]:
frame.to_hdf('mydata.h5', 'obj3', foramt='table')
pd.read_hdf('mydata.h5', 'obj3')

,a
0,0.550216
1,1.885966
2,-1.135988
3,2.351502
4,-0.748439
...,...
95,-0.057352
96,0.454569
97,1.610045
98,0.394329


注意：HDF5不是数据库。它最适合用作“一次写多次读”的数据集。虽然数据可以在任何时候被添加到文件中，但如果同时发生多个写操作，文件就可能会被破坏。

# 读取Microsoft Excel文件

pandas的ExcelFile类或pandas.read_excel函数支持读取存储在Excel 2003（或更高版本）中的表格型数据。这两个工具分别使用扩展包xlrd和openpyxl读取XLS和XLSX文件。你可以用pip或conda安装它们。

要使用**ExcelFile**，通过传递xls或xlsx路径创建一个实例：

存储在表单sheet中的数据可以**read_excel**读取到DataFrame

如果要读取一个文件中的多个表单，创建ExcelFile会更快，但你也可以将文件名传递到pandas.read_excel：

如果要将pandas数据写入为Excel格式，你必须首先创建一个ExcelWriter，然后使用pandas对象的**to_excel**方法将数据写入到其中：【该对象还要调用save方法】

你还可以不使用ExcelWriter，而是传递文件的路径到to_excel：

In [31]:
xlsx = pd.ExcelFile('examples/ex1.xlsx')
xlsx

In [32]:
 pd.read_excel(xlsx, 'Sheet1')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [33]:
frame = pd.read_excel('examples/ex1.xlsx', 'Sheet1')
frame

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [34]:
writer = pd.ExcelWriter('pdexcelwirte.xlsx')
frame.to_excel(writer,'sheet1')
writer.save()

In [ ]:
 frame.to_excel('pdexcelwirte.xlsx')

# 6.3 Web APIs交互-----------------------------------------------------------------------------------

许多网站都有一些通过JSON或其他格式提供数据的公共API。通过Python访问这些API的办法有不少。一个简单易用的办法（推荐）是requests包

In [ ]:
import requests
url = 'https://api.github.com/repos/pandas-dev/pandas/issues'
resp = requests.get(url)
data = resp.json()
issues = pd.DataFrame(data, columns=['number', 'title','labels','state'])
issues.to_csv('tempissues.csv')

# 6.4 数据库交互--------------------------------------------------------------------------------------

在商业场景下，大多数数据可能不是存储在文本或Excel文件中。基于SQL的关系型数据库（如SQL Server、PostgreSQL和MySQL等）使用非常广泛，其它一些数据库也很流行。数据库的选择通常取决于性能、数据完整性以及应用程序的伸缩性需求。

将数据从SQL加载到DataFrame的过程很简单，此外pandas还有一些能够简化该过程的函数。